In [41]:
import cv2
import pytesseract
import numpy as np
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import re

In [ ]:
!pip install transformers torch torchvision Pillow datasets

In [43]:
from PIL import Image
from torch.nn.utils.rnn import pad_sequence
import torch

In [44]:
image_path = "/content/drive/MyDrive/human_ai_folder/testing_folder/Mendo-page_5.jpg"
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [45]:
data = pytesseract.image_to_data(gray, output_type=pytesseract.Output.DICT)

In [ ]:
for i in range(len(data['text'])):
    if int(data['conf'][i]) > 0:
        x, y, w, h = data['left'][i], data['top'][i], data['width'][i], data['height'][i]
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

plt.figure(figsize=(10, 10))
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

In [ ]:
!pip install difflib

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

In [ ]:
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

In [ ]:
from torch.utils.data import Dataset
from transformers import VisionEncoderDecoderModel, TrOCRProcessor
from PIL import Image
import torch

In [ ]:
checkpoint_path = "D:\OCR_final\OCR\ocr_model\checkpoint"

In [ ]:
model = VisionEncoderDecoderModel.from_pretrained(checkpoint_path)
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
model.eval()

In [ ]:
def process_text_regions(image, data):
    extracted_text = ""

    for i in range(len(data['text'])):
        text = data['text'][i].strip()
        if text:
            x, y, w, h = data['left'][i], data['top'][i], data['width'][i], data['height'][i]
            cropped_region = image[y:y+h, x:x+w]
            pil_img = Image.fromarray(cropped_region).convert("RGB")

            pixel_values = processor(pil_img, return_tensors="pt").pixel_values
            generated_ids = model.generate(pixel_values)
            decoded_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

            extracted_text += decoded_text + " "

    return extracted_text

In [ ]:
final_text = process_text_regions(gray, data)
print("Extracted Text:")
print(final_text)

The channel dimension is ambiguous. Got image shape (3, 5, 3). Assuming channels are the first dimension.


Extracted Text:
VIDEATUR mor CHARAGE. NO AY ON TODO EL LIBRO COTA CONTRA NUEITTA SANTA FE CATHOLICA, DI QUE DEEDIGADE 10 MORAL CR LAS COFTURBERS : CONQUE DEBE DARFELE IA LICENCLA,QUE DIDE. EN EFFE COLLEGIO DEL ARZOLIPO DE TOLEDO MI SERIOR,EL MAYOR DE SALAMANCA, DULIO 3.DE 1656 DOCTOR DON FRANCIFCO DE PWGA,Y FEYOO. LICENCIA DEL ORDINARIO. ON LUAN PEREZ DELGADO POR LA GRACIA DE DIOS,Y DE LA FANTA SEDE A POFTOLICA OBIFPO DE SALAMANCA,DEL CONTEJO DE fu MAGETTAD,&C.POR LA PRECENTE, POR IO QUE 1 NOS TOCA,DAMOS LICENCIA, PARA QUE TE PUEDA IMPRIMIR EL LIBRO INTICULARO PRINCIPE PERFETTO,Y MINIFROS AJAftADOS, DOCMMENOS POLITICOS,Y MORALES, COPUEFFO POR EL P.ANDRES MENDO DE LA COMPANIA DE LEFUS, RECTOR DEL COLLEGIO DC IRIZLEGES DE 1 VINIUERRIAD DEFTA DICHA CLUDADJATEN- TO, DE 1A CENTURA DEL DOCTOR DON FRANCIFCO DE PUGA, Y FEIJO,COLEGIAL DEL MAYOR DEL ARZOBIFPO DE TOLEDO DE- TA DICHA VINIUERFIDAD, Y CATBEDRACICO DE PRIMA DE CA- HONES DELIA,CONFRA, NO TIENE COFA ALGUNA CONTA MUELRA FANTA FE CATHOLI

In [ ]:
def normalize_text(text):
      text = text.replace("ſ", "s") 
      text = text.replace("ç", "z")  
      text = text.replace("q̄", "que") 
      text = text.replace("u", "v") if "v" in text else text.replace("v", "u")  
      text = text.translate(str.maketrans("áéíóú", "aeiou")) 
      return text

In [ ]:
transcript_path = "D:\OCR_final\EvalText\Ezacary\page_1_cleaned.txt"
with open(transcript_path, "r", encoding="utf-8") as f:
    original_text = f.read().strip()

original_text=normalize_text(original_text)

In [ ]:
print("\nGround Truth Transcript:\n", original_text)

In [ ]:
!pip install spacy
!python -m spacy download es_core_news_sm

In [ ]:
import re
import spacy
nlp = spacy.load("es_core_news_sm")

def correct_case(text):
    text = text.lower()
    doc = nlp(text)

    corrected_text = " ".join(
        [sent.text.capitalize() for sent in doc.sents]
    )

    return corrected_text

extracted_text = final_text 
clean_text = correct_case(extracted_text)
print(clean_text)

Videatur mor charage. No ay on todo el libro cota contra nueitta santa fe catholica, di que deedigade 10 moral cr las cofturbers : conque debe darfele ia licencla,que dide. En effe collegio del arzolipo de toledo mi serior,el mayor de salamanca, dulio 3.de 1656 doctor don francifco de pwga,y feyoo. Licencia del ordinario. On luan perez delgado por la gracia de dios,y de la fanta sede a poftolica obifpo de salamanca,del contejo de fu magettad,&c.por la precente, por io que 1 nos toca,damos licencia, para que te pueda imprimir el libro inticularo principe perfetto,y minifros ajaftados, docmmenos politicos,y morales, copueffo por el p.andres mendo de la compania de lefus, rector del collegio dc irizleges de 1 viniuerriad defta dicha cludadjaten- to, de 1a centura del doctor don francifco de puga, y feijo,colegial del mayor del arzobifpo de toledo de- ta dicha viniuerfidad, y catbedracico de prima de ca- hones delia,confra, no tiene cofa alguna conta muelra fanta fe catholica, y buenas cof

In [ ]:
!pip install Levenshtein jiwer

## Evaluation:

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from difflib import SequenceMatcher

In [ ]:
def character_f1(ground_truth, ocr_text):
    ground_truth = ground_truth.lower().replace(" ", "")
    ocr_text = ocr_text.lower().replace(" ", "")

    char_set = set(ground_truth) | set(ocr_text)
    gt_vector = [1 if c in ground_truth else 0 for c in char_set]
    ocr_vector = [1 if c in ocr_text else 0 for c in char_set]

    f1 = f1_score(gt_vector, ocr_vector, average='binary')
    return f1

In [ ]:
original_text = original_text
ocr_text = clean_text

f1_char = character_f1(original_text, ocr_text)
print("Character-Level F1-score:", f1_char)

Character-Level F1-score: 0.8461538461538461


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
from collections import Counter

def word_f1(ground_truth, ocr_text):
    gt_words = ground_truth.lower().split()
    ocr_words = ocr_text.lower().split()

    gt_counter = Counter(gt_words)
    ocr_counter = Counter(ocr_words)

    common_words = sum((gt_counter & ocr_counter).values())
    precision = common_words / sum(ocr_counter.values()) if sum(ocr_counter.values()) > 0 else 0
    recall = common_words / sum(gt_counter.values()) if sum(gt_counter.values()) > 0 else 0

    if precision + recall == 0:
        return 0 
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

f1_word = word_f1(original_text, ocr_text)
print("Word-Level F1-score:", f1_word)

Word-Level F1-score: 0.45037037037037037


In [ ]:
image_path = "OCR/training_file/processed_images/Buendia-page_1.jpg"
image = cv2.imread(image_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [61]:
data = pytesseract.image_to_data(gray, output_type=pytesseract.Output.DICT)

In [62]:
final_text = process_text_regions(gray, data)
print("Extracted Text:")
print(final_text)

Extracted Text:
ICS PER VOS : DULCISIMO NINO : 3 1 JESUS : QUE NO TOLO OS : GSTRA 3 DIGNAFEIS DE LLAMARDS DOCTOR DE LOS NINOS, 1 INO TAMBIEN DE AISIF- CIR COMO NINO ENTRE LOS DOCTORES, TE CONFAGRA HUMILDE ETTA PEQUENA INSTRUCTION DE LOS NINOS. ES AISI, QUE ELLA TAMBIEN TE DIRICE 2 LA JU- VENTUD : PERO AEFTA, COMO RECUER- DO DE 10 QUE APRENDIO, ALOS NI- NOS > COMO PRECIFA EXPLICACION DE 10 QUE DEBEN EFTUDIAR. POR EFTE TO- IO TITULO.ES MUY VUEFRA > Y POR TER PARA NINOS , QUE CONFAIS 2 LA EDUCACION DE VUEFRA COMPANIA, 10 CS MUCHO MAS. EN VOS, ( DIVI- NO EXEMPLAR DE TODAS LAS VIRTU- DES ) ) TIENEN ABREVIADO CL MAS TE- 


In [ ]:
extracted_text = final_text
clean_text = correct_case(extracted_text)
print(clean_text)

Ics per vos : dulcisimo nino : 3 1 jesus : que no tolo os : gstra 3 dignafeis de llamards doctor de los ninos, 1 ino tambien de aisif- cir como nino entre los doctores, te confagra humilde etta pequena instruction de los ninos. Es aisi, que ella tambien te dirice 2 la ju- ventud : pero aefta, como recuer- do de 10 que aprendio, alos ni- nos > como precifa explicacion de 10 que deben eftudiar. Por efte to- io titulo.es muy vuefra > y por ter para ninos , que confais 2 la educacion de vuefra compania, 10 cs mucho mas. En vos, ( divi- no exemplar de todas las virtu- des ) ) tienen abreviado cl mas te-


In [ ]:
transcript_path = "D:\OCR_final\OCR\training_file\transcript\transcript_Buendia\page1.txt"
with open(transcript_path, "r", encoding="utf-8") as f:
    original_text = f.read().strip()


original_text=normalize_text(original_text)
print(original_text)

In [65]:
print(f"\nCER (Character Error Rate): {cer:.4f}")
print(f"WER (Word Error Rate): {wer_value:.4f}")


CER (Character Error Rate): 0.2427
WER (Word Error Rate): 0.7857


In [ ]:
f1_char = character_f1(original_text, ocr_text)
print("Character-Level F1-score:", f1_char)

Character-Level F1-score: 0.8461538461538461


In [ ]:
f1_word = word_f1(original_text, ocr_text)
print("Word-Level F1-score:", f1_word)

Word-Level F1-score: 0.45037037037037037
